# YOLO based models

# HerdNet

In [1]:
# from PytorchWildlife.models import detection as pw_detection
import pandas as pd
from pathlib import Path
import yaml
import os
from tqdm import tqdm
from PIL import Image

In [7]:
# herdnet_path = r"C:\Users\Machine Learning\Desktop\workspace-wildAI\datalabeling\base_models_weights\20220329_HerdNet_Ennedi_dataset_2023.pth"
# herdnet_detector = pw_detection.HerdNet(weights=herdnet_path,device="cuda")
# herdnet_detector.single_image_detection(img=r"D:\PhD\Data per camp\DetectionDataset\Rep 1\train\images\DJI_20231001142304_0082_0_51_2160_0_3440_1280.jpg",det_conf_thres=0.25,)

In [ ]:
gt_csv = r"D:\PhD\Data per camp\DetectionDataset\Rep 1\train\gt.csv"

def save_non_empty_gt(gt_csv:str):
    
    # images,x,y,labels

    df_gt = pd.read_csv(gt_csv)
    df_gt = df_gt.dropna(axis=0,how="any")
    df_gt.loc[:,'label_id'] = df_gt['label_id'].apply(int)
    df_gt = df_gt.astype({'label_id':'int64'})

    df_gt.rename(columns={'labels':'label_names','label_id':'labels'},
                 inplace=True)
    
    # df_gt.drop(columns=['x0','x1','y0','y1'],inplace=True)

    save_path = Path(gt_csv).with_stem("gt_nonempty")
    df_gt.to_csv(save_path,
                  index=False,
                )

    print(f"Saving non-empty gt.csv to {save_path}.")
    return df_gt, str(save_path)

In [2]:
def check_label_format(loaded_df:pd.DataFrame)->str:
    """checks label format

    Args:
        loaded_df (pd.DataFrame): target values

    Raises:
        NotImplementedError: when the format is not yolo or yolo-obb

    Returns:
        str: yolo or yolo-obb
    """

    num_features = len(loaded_df.columns)

    if num_features == 5:
        return "yolo"
    elif num_features == 9:
        return "yolo-obb"
    else:
        raise NotImplementedError(f"The number of features ({num_features}) in the label file is wrong. Check yolo or yolo-obb format from ultralytics.")


def get_groundtruth(yolo_labels_dir, save_path:str=None,load_gt_csv:str=None):

    if load_gt_csv is not None:
        return pd.read_csv(load_gt_csv)

    cols1 = ['id','x1','y1','x2','y2','x3','y3','x4','y4']
    cols2 = ['id','x','y','w','h']

    if not Path(yolo_labels_dir).exists():
        raise FileNotFoundError('Directory does not exist.')

    # Iterate through labels
    dfs = list()
    for label_path in tqdm(Path(yolo_labels_dir).glob("*.txt"),desc="Getting groundtruths"):
        df = pd.read_csv(label_path,sep=' ',header=None)

        image_path = Path(str(label_path).replace('labels','images')).with_suffix(".jpg")
        img = Image.open(image_path)
        img_width, img_height = img.size

        label_format = check_label_format(df)

        if label_format == 'yolo':
            df.columns = cols2
            df.loc[:,'x'] = df['x']*img_width
            df.loc[:,'y'] = df['y']*img_height
            df.loc[:,'w'] = df['w']*img_width
            df.loc[:,'h'] = df['h']*img_height
        else: # yolo-obb
            df.columns = cols1
            df['x'] = (df['x1'] + df['x2'])*img_width*0.5
            df['y'] = (df['y1'] + df['y4'])*img_height*0.5
            df['w'] = (df['x2'] - df['x1'])*img_width
            df['h'] = (df['y4'] - df['y1'])*img_height
        df['images'] = str(image_path)
        df.rename(columns={'id':'labels'}, inplace=True)
        dfs.append(df)
        img.close()
    
    dfs = pd.concat(dfs)
    if save_path is not None:
        dfs.to_csv(save_path, index=False)

    return dfs

In [ ]:
import albumentations as A
from torch.utils.data import ConcatDataset
from animaloc.datasets import CSVDataset
from animaloc.data.transforms import MultiTransformsWrapper, DownSample, PointsToMask, FIDT

def load_dataset(data_config_yaml:str,
                 split:str='train',
                 transforms:dict=None,
                 down_ratio:int=2,
                 num_classes = 6,
                 patch_size:int=800):

    
    if transforms is None:
        transforms = {}
        transforms['train'] = ([    #A.Resize(patch_size,patch_size,p=1.0),
                                    A.VerticalFlip(p=0.5), 
                                    A.HorizontalFlip(p=0.5),
                                    A.RandomRotate90(p=0.5),
                                    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.2),
                                    A.Blur(blur_limit=15, p=0.2),
                                    A.Normalize(p=1.0)
                                ], 
                                [
                                    MultiTransformsWrapper([
                                    FIDT(num_classes=num_classes, down_ratio=down_ratio),
                                    PointsToMask(radius=2, 
                                                 num_classes=num_classes, 
                                                 squeeze=True, 
                                                 down_ratio=int(patch_size//(16*patch_size/512))
                                                )
                                ])
                                ])
        transforms['val'] = (
                                [#A.Resize(patch_size,patch_size,p=1.0),
                                 A.Normalize(p=1.0)],
                                [DownSample(down_ratio=down_ratio, anno_type='point')]
                            )
        transforms['test'] = transforms['val']
    

    with open(data_config_yaml,'r') as file:
        data_config = yaml.load(file,Loader=yaml.FullLoader)
    datasets = list()
    df_gts = list()
    root = data_config['path']
    for data in tqdm(data_config[split],desc="concatenating datasets"):
        img_dir = os.path.join(root,data)
        # path_to_csv = Path(img_dir).parent/"gt.csv"
        df = get_groundtruth(yolo_labels_dir=img_dir.replace("images","labels"), 
                             save_path=None,
                             load_gt_csv=None # path_to_csv
                            )        
        dataset = CSVDataset(
                            csv_file = df,
                            root_dir = img_dir,
                            albu_transforms = transforms[split][0],
                            end_transforms = transforms[split][1]
                            )
        datasets.append(dataset)
        df_gts.append(df)
    
    return ConcatDataset(datasets=datasets), pd.concat(df_gts)

c:\Users\Machine Learning\anaconda3\envs\label-backend\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Machine Learning\anaconda3\envs\label-backend\Lib\site-packages\albumentations\__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.5' (you have '2.0.4'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [ ]:
train_dataset, df_train_labels = load_dataset(data_config_yaml=r"C:\Users\Machine Learning\Desktop\workspace-wildAI\datalabeling\data\dataset_identification.yaml",
                            split='train',
                            down_ratio=2,
                            transforms=None,
                            num_classes=6,
                            patch_size=800
                        )

Getting groundtruths: 4984it [00:07, 654.10it/s]:00<?, ?it/s]
concatenating datasets: 100%|██████████| 1/1 [00:08<00:00,  8.23s/it]


In [5]:
df_train_labels_freq = df_train_labels['labels'].value_counts().sort_index()/len(df_train_labels)
df_train_labels_freq

labels
0     0.001721
1     0.056920
2     0.000536
3     0.001568
4     0.004629
5     0.136026
6     0.011323
7     0.040777
8     0.005929
9     0.022875
10    0.005623
11    0.011093
12    0.058680
13    0.008798
14    0.352689
15    0.192487
16    0.024061
17    0.023066
18    0.041198
Name: count, dtype: float64

In [6]:
val_dataset, df_val_labels = load_dataset(data_config_yaml=r"C:\Users\Machine Learning\Desktop\workspace-wildAI\datalabeling\data\dataset_identification.yaml",
                            split='val',
                            down_ratio=2,
                            transforms=None,
                            num_classes=19,
                            patch_size=1280
                        )

# test_dataset = CSVDataset(
#     csv_file = '/content/data/test.csv',
#     root_dir = '/content/data/test',
#     albu_transforms = [A.Normalize(p=1.0)],
#     end_transforms = [DownSample(down_ratio=down_ratio, anno_type='point')]
#     )

Getting groundtruths: 899it [00:01, 600.41it/s]0:00<?, ?it/s]
concatenating datasets: 100%|██████████| 1/1 [00:01<00:00,  1.57s/it]


In [7]:
# for a in train_dataset:
#     print(a)
#     break

In [8]:
# a[1][1].shape

In [17]:
# Dataloaders
from torch.utils.data import DataLoader

train_dataloader = DataLoader(dataset = train_dataset, batch_size = 16, shuffle = True)

val_dataloader = DataLoader(dataset = val_dataset, batch_size = 16, shuffle = False)

# test_dataloader = DataLoader(dataset = test_dataset, batch_size = 1, shuffle = False)

In [18]:
# compute loss weigths
weights = 1/(df_train_labels_freq + 1e-6)
weights = weights.to_numpy()
# weights = weights.clip(min=1.,max=200)
weights

array([      580.6,      17.568,      1863.8,       637.2,         216,      7.3515,       88.31,      24.523,      168.63,      43.714,      177.81,      90.137,      17.041,      113.65,      2.8353,      5.1951,      41.559,      43.351,      24.272])

In [19]:
# number of classes
weights.shape[0]

19

In [20]:
from animaloc.models import HerdNet
import torch
from torch import Tensor
from animaloc.models import LossWrapper
from animaloc.train.losses import FocalLoss
from torch.nn import CrossEntropyLoss

num_classes = weights.shape[0]
down_ratio = 2

herdnet = HerdNet(num_classes=num_classes, down_ratio=down_ratio, pretrained=False)
herdnet_path = r"C:\Users\Machine Learning\Desktop\workspace-wildAI\datalabeling\base_models_weights\20220329_HerdNet_Ennedi_dataset_2023.pth"
checkpoint = torch.load(herdnet_path)
herdnet.load_state_dict(checkpoint['model_state_dict'],strict=False)

# herdnet = pw_detection.HerdNet(weights=herdnet_path,device="cuda").model  

# move to device
herdnet = herdnet.to('cuda')
weights = Tensor(weights).to('cuda')

losses = [
    {'loss': FocalLoss(reduction='mean'), 'idx': 0, 'idy': 0, 'lambda': 1.0, 'name': 'focal_loss'},
    {'loss': CrossEntropyLoss(reduction='mean', weight=weights), 'idx': 1, 'idy': 1, 'lambda': 1.0, 'name': 'ce_loss'}
    ]

herdnet = LossWrapper(herdnet, losses=losses)

In [21]:
!wandb login
!wandb offline

W&B offline. Running your script from this directory will only write metadata locally. Use wandb disabled to completely turn off W&B.


In [ ]:
from torch.optim import Adam
from animaloc.train import Trainer
from animaloc.eval import PointsMetrics, HerdNetStitcher, HerdNetEvaluator

work_dir = '../.tmp'

lr = 1e-4
weight_decay = 1e-3
epochs = 30
patch_size = 800

optimizer = Adam(params=herdnet.parameters(), lr=lr, weight_decay=weight_decay)

metrics = PointsMetrics(radius=20, num_classes=num_classes)

stitcher = HerdNetStitcher(
    model=herdnet, 
    size=(patch_size,patch_size), 
    batch_size=1,
    overlap=160, 
    down_ratio=down_ratio, 
    reduction='mean'
    )

evaluator = HerdNetEvaluator(
    model=herdnet, 
    dataloader=val_dataloader, 
    metrics=metrics, 
    stitcher=stitcher, 
    work_dir=work_dir, 
    header='validation'
    )

trainer = Trainer(
    model=herdnet,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    lr_milestones=[20,],
    optimizer=optimizer,
    auto_lr=True,
    num_epochs=epochs,
    evaluator=evaluator,
    work_dir=work_dir
    )

In [ ]:
# herdnet = trainer.resume(pth_path=herdnet_path,select='max',validate_on='recall')
# trainer.model = herdnet.cuda()

In [24]:
herdenet = trainer.start(warmup_iters=10, checkpoints='best', select='max', validate_on='f1_score')

[TRAINING] - Epoch: [1] [  1/312] eta: 14:51:28 lr: 0.000002 loss: 80600.1016 (80600.1016) focal_loss: 80597.1797 (80597.1797) ce_loss: 2.9197 (2.9197) time: 171.4360 data: 2.9852 max mem: 51244
[TRAINING] - Epoch: [1] [  1/312] eta: 14:51:28 lr: 0.000002 loss: 80600.1016 (80600.1016) focal_loss: 80597.1797 (80597.1797) ce_loss: 2.9197 (2.9197) time: 171.4360 data: 2.9852 max mem: 51244


[TRAINING] - Epoch: [1] [  1/312] eta: 14:51:28 lr: 0.000002 loss: 80600.1016 (80600.1016) focal_loss: 80597.1797 (80597.1797) ce_loss: 2.9197 (2.9197) time: 171.4360 data: 2.9852 max mem: 51244


KeyboardInterrupt: 

# PaddlePaddle

# MMROTATE